<a href="https://colab.research.google.com/github/Nazaruk-Anton/Image_Classification_with_Pre-trained_Models/blob/main/2_Pretraining_%26_Comparing_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Libraries, Packages, and Data

In [1]:
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

In [13]:
## get the data
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

--2023-07-02 11:21:24--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261483817 (249M) [application/zip]
Saving to: ‘concrete_data_week4.zip’

concrete_data_week4 100%[===================>] 249.37M  39.0MB/s    in 6.6s    

2023-07-02 11:21:31 (37.8 MB/s) - ‘concrete_data_week4.zip’ saved [261483817/261483817]



In [14]:
!unzip -q concrete_data_week4.zip

## Define Global Constants & Construct ImageDataGenerator Instances

In [15]:
# Define the number of classes and image resize dimensions

num_classes = 2

image_resize = 224

batch_size_training = 100
batch_size_validation = 100

In [17]:
# Create an ImageDataGenerator instance for preprocessing the data

data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

# Create a generator for training data

train_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

# Create a generator for validation data

validation_generator = data_generator.flow_from_directory(
    "concrete_data_week4/valid",
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode="categorical",
)

# Create a generator for testing data

testing_generator = data_generator.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.
Found 9501 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


## Build, Compile and Fit Model ResNet50

In [18]:
# Build the model architecture using ResNet50 as the base model

model = Sequential()

model.add(ResNet50(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

model.add(Dense(num_classes, activation='softmax'))

model.layers

In [19]:
# Freeze the weights of the ResNet50 base model

model.layers[0].trainable = False

In [20]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense_2 (Dense)             (None, 2)                 4098      
                                                                 
Total params: 23,591,810
Trainable params: 4,098
Non-trainable params: 23,587,712
_________________________________________________________________


In [21]:
# Compile the model

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define the number of steps per epoch for training and validation

steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

In [ ]:
# Fit the model to the training data

fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

<ipython-input-11-de8d7382ad83>:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  fit_history = model.fit_generator(


Epoch 1/2


In [10]:
# Save the trained model

model.save('classifier_resnet_model.h5', include_optimizer=False)

##Build, Compile and Fit Model VGG16

In [22]:
# Build the model architecture using VGG16 as the base model

model_vgg16 = Sequential()

model_vgg16.add(VGG16(include_top=False, pooling="avg", weights="imagenet",))
model_vgg16.add(Dense(num_classes, activation="softmax"))

# Freeze the weights of the VGG16 base model
model_vgg16.layers[0].trainable = False

model_vgg16.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 dense_3 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [23]:
# Compile the model

model_vgg16.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)

# Define the number of steps per epoch for training and validation

num_epochs = 2
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)

In [ ]:
# Fit the model to the training data

history_vgg16 = model_vgg16.fit_generator(
    train_generator ,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

58889256/58889256 [==============================] - 0s 0us/step
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 dense_1 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


<ipython-input-8-f6851d96f1ef>:18: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history_vgg16 = model_vgg16.fit_generator(


Epoch 1/2
176/301 [================>.............] - ETA: 2:06:08 - loss: 0.1220 - accuracy: 0.9610

In [ ]:
# Save the trained model

model_vgg16.save("classifier_vgg16_model.h5")

## Model Comparison

In [24]:
# Load the saved ResNet50 and VGG16 models

from keras.models import load_model

model_resnet50 = load_model("/content/classifier_resnet_model.h5")
model_vgg16 = load_model("/content/classifier_vgg16_model.h5")


In [25]:
# Evaluate the performance of the VGG16 model

performance_vgg16 = model_vgg16.evaluate_generator(testing_generator)
print("Performance of the VGG16-trained model")
print("Loss: {}".format(round(performance_vgg16[0], 5)))
print("Accuracy: {}".format(round(performance_vgg16[1], 5)))

<ipython-input-25-b011ab8e0084>:3: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  performance_vgg16 = model_vgg16.evaluate_generator(testing_generator)


Performance of the VGG16-trained model
Loss: 0.01861
Accuracy: 0.996


In [26]:
# Evaluate the performance of the ResNet50 model

performance_resnet50 = model_resnet50.evaluate_generator(testing_generator)
print("Performance of the ResNet50-trained model")
print("Loss: {}".format(round(performance_resnet50[0], 5)))
print("Accuracy: {}".format(round(performance_resnet50[1], 5)))

<ipython-input-26-df9b3b418bf3>:3: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  performance_resnet50 = model_resnet50.evaluate_generator(testing_generator)


Performance of the ResNet50-trained model
Loss: 0.27634
Accuracy: 0.882


In [27]:
# Generate predictions using the VGG16 model

predictions_vgg16 = model_vgg16.predict_generator(testing_generator, steps=1)

# Function to print the prediction result

def print_prediction(prediction):
    if prediction[0] > prediction[1]:
        print("Negative ({}% certainty)".format(round(prediction[0] * 100, 1)))
    elif prediction[1] > prediction[0]:
        print("Positive ({}% certainty)".format(round(prediction[1] * 100, 1)))
    else:
        print("Unsure (prediction split 50–50)")

# Print the first five predictions for the VGG16 model

print("First five predictions for the VGG16-trained model:")
for i in range(5):
    print_prediction(predictions_vgg16[i])

<ipython-input-27-10bcf767c9df>:3: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions_vgg16 = model_vgg16.predict_generator(testing_generator, steps=1)


First five predictions for the VGG16-trained model:
Negative (99.9% certainty)
Positive (100.0% certainty)
Positive (99.7% certainty)
Negative (95.4% certainty)
Negative (99.4% certainty)


In [28]:
# Generate predictions using the ResNet50 model

predictions_resnet50 = model_resnet50.predict_generator(testing_generator, steps=1)
print("First five predictions for the ResNet50-trained model:")
for i in range(5):
    print_prediction(predictions_resnet50[i])


<ipython-input-28-3e9d47e1eae1>:3: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions_resnet50 = model_resnet50.predict_generator(testing_generator, steps=1)


First five predictions for the ResNet50-trained model:
Positive (81.9% certainty)
Negative (100.0% certainty)
Positive (97.5% certainty)
Negative (100.0% certainty)
Negative (100.0% certainty)
